In [1]:
import tensorflow as tf

2022-07-23 20:03:29.758021: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-23 20:03:29.758068: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/opt/conda/lib/python3.7/site-packages/jax/__init__.py:27: UserWarning: cloud_tpu_init failed: KeyError('<!DOCTYPE html>\n<html lang=en>\n  <meta charset=utf-8>\n  <meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width">\n  <title>Error 404 (Not Found)!!1</title>\n  <style>\n    *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 2

In [4]:
tf.__version__

'2.8.0'

In [5]:
tfr = 'gs://jk-t5x-staging/experiments/fine-tune-t5-xl-v2-8/dataset/wmt_t2t_translate/de-en/1.0.0/wmt_t2t_translate-train.tfrecord-00000-of-00016'

In [8]:
raw_dataset = tf.data.TFRecordDataset([tfr])

In [9]:
raw_dataset

<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [10]:
for raw_record in raw_dataset.take(2):
    print(repr(raw_record))

<tf.Tensor: shape=(), dtype=string, numpy=b'\n\x8f\x02\ny\n\x02en\x12s\nq\noHere the largest town of the district is located: Nordenham , lying opposite to Bremerhaven at the Weser mouth.\n\x91\x01\n\x02de\x12\x8a\x01\n\x87\x01\n\x84\x01An der B 211 befindet sich in Loyermoor der so genannte \xe2\x80\x9eGeest-Abbruch\xe2\x80\x9c, der eine H\xc3\xb6hendifferenz von gut 30 Meter \xc3\xbcberbr\xc3\xbcckt.'>
<tf.Tensor: shape=(), dtype=string, numpy=b"\n\xd1\x03\n\xd8\x01\n\x02de\x12\xd1\x01\n\xce\x01\n\xcb\x01Ich begr\xc3\xbc\xc3\x9fe die Erkl\xc3\xa4rung des Herrn Kommissar und die Arbeit des Parlaments, die wirklich deutlich macht, welch konkretes Interesse wir an diesen Fragen haben, gerade unter den derzeitigen Umst\xc3\xa4nden.\n\xf3\x01\n\x02en\x12\xec\x01\n\xe9\x01\n\xe6\x01I should like, in passing, to pay tribute to the Commissioner' s statement and to all the work that Parliament has carried out, which shows that we attach real importance to these issues, particularly in the cur

In [11]:
feature_description = {
    'en': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'de': tf.io.FixedLenFeature([], tf.string, default_value='')
}

In [12]:
def _parse_function(example_proto):
  # Parse the input `tf.train.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_description)

In [13]:
parsed_dataset = raw_dataset.map(_parse_function)
parsed_dataset

<MapDataset element_spec={'de': TensorSpec(shape=(), dtype=tf.string, name=None), 'en': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [14]:
for parsed_record in parsed_dataset.take(2):
  print(repr(parsed_record))

{'de': <tf.Tensor: shape=(), dtype=string, numpy=b'An der B 211 befindet sich in Loyermoor der so genannte \xe2\x80\x9eGeest-Abbruch\xe2\x80\x9c, der eine H\xc3\xb6hendifferenz von gut 30 Meter \xc3\xbcberbr\xc3\xbcckt.'>, 'en': <tf.Tensor: shape=(), dtype=string, numpy=b'Here the largest town of the district is located: Nordenham , lying opposite to Bremerhaven at the Weser mouth.'>}
{'de': <tf.Tensor: shape=(), dtype=string, numpy=b'Ich begr\xc3\xbc\xc3\x9fe die Erkl\xc3\xa4rung des Herrn Kommissar und die Arbeit des Parlaments, die wirklich deutlich macht, welch konkretes Interesse wir an diesen Fragen haben, gerade unter den derzeitigen Umst\xc3\xa4nden.'>, 'en': <tf.Tensor: shape=(), dtype=string, numpy=b"I should like, in passing, to pay tribute to the Commissioner' s statement and to all the work that Parliament has carried out, which shows that we attach real importance to these issues, particularly in the current circumstances.">}
